In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import unicodecsv
import pandas as pd
from IPython.display import display, HTML
import numpy as np  
np.seterr(divide='ignore', invalid='ignore')

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)
    
Master = pd.read_csv('Master.csv')
Batting = pd.read_csv('Batting.csv')
Pitching = pd.read_csv('Pitching.csv')
Fielding= pd.read_csv('Fielding.csv')

In [ ]:
def batter_stats(player,stat,team="None",league="None",minyear=0,maxyear=3000):
    total = 0
    if_statement= "batter['playerID'] == player"
    for batter in Batting:
        if eval(if_statement):
            if team <> "None":
                if_statement += " and batter['teamID'] == team"
            if league <> "None":
                if_statement += " and batter['lgID'] == league"
            if minyear <> 0:
                if_statement += " and batter['yearID'] >= minyear"
            if maxyear <> 3000:
                if_statement += " and batter['yearID'] <= maxyear"
        if eval(if_statement):
            total = total + batter[stat]
    return total

def pitcher_stats(player="None",stat="H",team="None",league="None",minyear=0,maxyear=3000):
    total = 0
    if_statement= "pitcher['playerID'] == player"
    for pitcher in Pitching:
        if eval(if_statement):
            if team <> "None":
                if_statement += " and pitcher['teamID'] == team"
            if league <> "None":
                if_statement += " and pitcher['lgID'] == league"
            if minyear <> 0:
                if_statement += " and pitcher['yearID'] >= minyear"
            if maxyear <> 3000:
                if_statement += " and pitcher['yearID'] <= maxyear"
        if eval(if_statement):
            total = total + pitcher[stat]
    return total 

def fielder_stats(player="None",stat="H",team="None",league="None",minyear=0,maxyear=3000):
    total = 0
    if_statement= "fielder['playerID'] == player"
    for fielder in Fielding:
        if eval(if_statement):
            if team <> "None":
                if_statement += " and fielder['teamID'] == team"
            if league <> "None":
                if_statement += " and fielder['lgID'] == league"
            if minyear <> 0:
                if_statement += " and fielder['yearID'] >= minyear"
            if maxyear <> 3000:
                if_statement += " and fielder['yearID'] <= maxyear"
        if eval(if_statement):
            total = total + fielder[stat]
    return total 

In [ ]:
def batter_PCs(player,team="None",league="None",minyear=0,maxyear=3000):
    totalAB = batter_stats(player,"AB",team,league,minyear,maxyear)
    totalH = batter_stats(player,"H",team,league,minyear,maxyear)
    totalBB = batter_stats(player,"BB",team,league,minyear,maxyear)
    total2B = batter_stats(player,"2B",team,league,minyear,maxyear)
    total3B = batter_stats(player,"3B",team,league,minyear,maxyear)
    totalHR = batter_stats(player,"HR",team,league,minyear,maxyear)
    totalHBP = batter_stats(player,"HBP",team,league,minyear,maxyear)
    totalSF =  batter_stats(player,"SF",team,league,minyear,maxyear)
    OPS = 0
    TB = 0
    SLG = 0
    OBP = 0    
    TB = TB + (totalH - (total2B + total3B + totalHR)) + (np.multiply(2,total2B)) + (np.multiply(3,total3B)) +(np.multiply(4,totalHR))
    SLG = round(SLG + np.divide(TB,totalAB),3)
    OBP = round(OBP + np.divide((totalH  + totalHBP + totalBB), (totalAB + totalHBP + totalSF + totalBB)),3)
    OPS = round(np.add(SLG,OBP),3)
    AVG = round((totalH / totalAB),3)
    return AVG,SLG,OBP,OPS

def pitcher_PCs(player):
    totalHBB = 0
    totalER =  pitcher_stats(player,"ER",team,league,minyear,maxyear)
    totalSO =  pitcher_stats(player,"SO",team,league,minyear,maxyear)
    totalH = pitcher_stats(player,"H",team,league,minyear,maxyear)
    totalBB = pitcher_stats(player,"BB",team,league,minyear,maxyear)
    totalIPouts = pitcher_stats(player,"IPouts",team,league,minyear,maxyear)
    totalInn = 0
    WHIP = 0
    totalHBB = totalH + totalBB
    totalIPouts = totalIPouts + pitcher['IPouts']
    totalInn = np.divide(totalIPouts,3)
    ERA = round((totalER/totalInn),2)   
    KPI = round((totalSO /totalInn),2)
    KVSBB = round((totalSO/totalBB),2)
    WHIP = np.around(np.divide(totalHBB,totalInn),2)
    return ERA,KPI,KVSBB,WHIP 

def batters_stats_by_decade(stat,minyear,maxyear,team="None",league="None"):
    decade_total = 0
    decade_average = 0
    batter_count = 0
    maxval = 0
    minval = 99999
    if_statement= "batter['yearID'] >= minyear and batter['yearID'] <= maxyear and (batter['AB'] + batter['BB'] + batter['IBB'] + batter['HBP'] + batter['SH'] + batter['SF']) > 502"
    for batter in Batting:
        if eval(if_statement):
            batter_count += 1
            if team <> "None":
                if_statement += " and batter['teamID'] == team"
            if league <> "None":
                if_statement += " and batter['lgID'] == league"
        if eval(if_statement):
            if batter[stat] > maxval:
                maxval = batter[stat]
                maxbatter = batter['playerID']
            if batter[stat] < minval:
                minval = batter[stat]
                minbatter = batter['playerID']
            decade_total = decade_total + batter[stat]
            decade_average = np.divide(decade_total,batter_count)
    return decade_total,decade_average,maxbatter,maxval,minbatter,minval

def pitchers_stats_by_decade(stat,minyear,maxyear,team="None",league="None"):
    decade_total = 0
    decade_average = 0
    pitcher_count = 0
    minouts = 486
    maxval = 0
    minval = 99999
    if_statement= "pitcher['yearID'] >= minyear and pitcher['yearID'] <= maxyear and pitcher['IPouts'] >= minouts"
    for pitcher in Pitching:
        if eval(if_statement):
            pitcher_count += 1
            print pitcher['playerID']
            print pitcher['yearID']
            if team <> "None":
                if_statement += " and pitcher['teamID'] == team"
            if league <> "None":
                if_statement += " and pitcher['lgID'] == league"
        if eval(if_statement):
            if pitcher[stat] > maxval:
                maxval = pitcher[stat]
                maxpitcher = pitcher['playerID']
            if batter[stat] < minval:
                minval = pitcher[stat]
                minpitcher = pitcher['playerID']
            decade_total = decade_total + pitcher[stat]
            decade_average = np.divide(decade_total,pitcher_count)
    return decade_total,decade_average,maxpitcher,maxval,minpitcher,minval

In [ ]:
def player_weight(player):
    for players in Master:
        if (players['playerID'] == player):
            return (players['weight'])
        
def getABperHR(player):
    HR = batter_stats(player,"HR")
    AB = batter_stats(player,"AB")
    abperhr = np.divide(AB,HR)
    return np.rint(abperhr)
        
def compare_weight_to_HR(player):
    ABperHR = getABperHR(player)
    weight = player_weight(player)
    return player, ABperHR, weight

def weight2HR_Add_to_dataframe(mydf):
    #count = 0
    for batter in Batting:
        #if batter['playerID'] == player:
        #count = count + 1
        playerID = batter['playerID']
        w2HR =  compare_weight_to_HR(playerID)
        mydf = mydf.append({'Player':w2HR[0], 'ABperHR':w2HR[1], 'Weight': w2HR[2]},ignore_index=True)
        #if count > 15: 
        #break
    mydf.drop_duplicates(subset = batter['playerID'], inplace = True)
    mydf.to_csv('w2hr.csv', index=False)
        